# <center> Garmin Watch Data 

In [34]:
import json
import pandas as pd
import os

os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')
import basic_methods as bm
import time

In [2]:
f = open('/Users/jayrajparmar/Documents/side_project/health_data_tracking/Garmin/jayrajparmar009@gmail.com_0_summarizedActivities.json', "r")
data = json.loads(f.read())

json_data = data[0]['summarizedActivitiesExport']

lis = []

for i in range(len(json_data)):
    df = pd.DataFrame([json_data[i]], columns = json_data[i].keys())
    lis.append(df)
df_garmin_exec = pd.concat(lis)


In [141]:
df_garmin_exec.head()

,activityId,uuidMsb,uuidLsb,name,activityType,userProfileId,timeZoneId,beginTimestamp,eventTypeId,rule,sportType,startTimeGmt,startTimeLocal,duration,distance,avgSpeed,avgHr,maxHr,maxRunCadence,steps,calories,bmrCalories,avgStrideLength,avgFractionalCadence,maxFractionalCadence,elapsedDuration,movingDuration,deviceId,avgDoubleCadence,maxDoubleCadence,summarizedDiveInfo,manufacturer,lapCount,waterEstimated,purposeful,autoCalcCalories,favorite,pr,elevationCorrected,decoDive,atpActivity,parent,elevationGain,elevationLoss,minElevation,maxElevation,maxVerticalSpeed,maxSpeed,startLongitude,startLatitude,vO2MaxValue,locationName,endLongitude,endLatitude,avgVerticalSpeed,floorsClimbed,floorsDescended,summarizedExerciseSets,activeSets,totalSets,totalReps,minRespirationRate,maxRespirationRate,avgRespirationRate,startStress,endStress,differenceStress
0,8959734435,-5634344091463760210,-5357954891567381561,Elliptical,elliptical,82576128,149,1654442920000,9,private,FITNESS_EQUIPMENT,"1,654,442,920,000.0000","1,654,428,520,000.0000","2,983,868.8965",0.0000,0.0000,148.0000,164.0000,79.0000,"2,258.0000","2,832.4535",318.4415,0.0000,0.1875,0.5000,"2,983,868.8965",0.0000,3317189130,48.3750,159.0000,{},GARMIN,1,298.0000,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8952433395,-1097609307350545764,-5078813204589093572,Cardio,indoor_cardio,82576128,149,1654352911000,9,private,FITNESS_EQUIPMENT,"1,654,352,911,000.0000","1,654,338,511,000.0000","144,330.9937",0.0000,0.0000,84.0000,89.0000,NaN,NaN,37.7102,12.5701,NaN,0.0000,0.0000,"144,330.9937",0.0000,3317189130,NaN,NaN,{},GARMIN,1,14.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-17,939.9994","-17,700.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8917024003,291049867221549316,-6719738246007706917,Cardio,indoor_cardio,82576128,149,1653836425000,9,private,TRAINING,"1,653,836,425,000.0000","1,653,822,025,000.0000","6,977,541.0156",0.0000,0.0000,121.0000,151.0000,NaN,NaN,"3,456.7665",745.8236,NaN,0.0000,0.0000,"6,977,541.0156",0.0000,3317189130,NaN,NaN,{},GARMIN,1,697.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-9,700.0000","-8,459.9998",0.1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8899731934,2752421229750862679,-6627922957386941904,Cardio,indoor_cardio,82576128,149,1653606038000,9,private,TRAINING,"1,653,606,038,000.0000","1,653,591,638,000.0000","3,719,052.9785",0.0000,0.0000,153.0000,188.0000,NaN,NaN,"3,230.5054",398.0519,NaN,0.0000,0.0000,"3,719,052.9785",0.0000,3317189130,NaN,NaN,{},GARMIN,1,371.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-3,179.9999","-3,179.9999",0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8896332721,-8383978575552557445,-4936977240004103301,Cardio,indoor_cardio,82576128,149,1653563124000,9,private,FITNESS_EQUIPMENT,"1,653,563,124,000.0000","1,653,548,724,000.0000","6,806,884.7656",0.0000,0.0000,136.0000,173.0000,NaN,NaN,"4,156.4998",729.0635,NaN,0.0000,0.0000,"6,806,884.7656",0.0000,3317189130,NaN,NaN,{},GARMIN,1,680.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-2,100.0000","-1,400.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
def duration_measure(col):
#     mili_sec = int(col)
    sec = int((col/1000))
    minutes = int((sec)/60)
    return minutes

In [70]:
df_garmin_imp_cols.head()

,activityId,name,activityType,startTimeLocal,duration,avgHr,maxHr,calories,bmrCalories,waterEstimated,datetime,date,duration_minutes
0,8959734435,Elliptical,elliptical,"1,654,428,520,000.0000",2983868,148.0000,164.0000,"2,832.4535",318.4415,298.0000,2022-06-05 07:28:40,2022-06-05,49
0,8952433395,Cardio,indoor_cardio,"1,654,338,511,000.0000",144330,84.0000,89.0000,37.7102,12.5701,14.0000,2022-06-04 06:28:31,2022-06-04,49
0,8917024003,Cardio,indoor_cardio,"1,653,822,025,000.0000",6977541,121.0000,151.0000,"3,456.7665",745.8236,697.0000,2022-05-29 07:00:25,2022-05-29,49
0,8899731934,Cardio,indoor_cardio,"1,653,591,638,000.0000",3719052,153.0000,188.0000,"3,230.5054",398.0519,371.0000,2022-05-26 15:00:38,2022-05-26,49
0,8896332721,Cardio,indoor_cardio,"1,653,548,724,000.0000",6806884,136.0000,173.0000,"4,156.4998",729.0635,680.0000,2022-05-26 03:05:24,2022-05-26,49


In [78]:
df_garmin_imp_cols = df_garmin_exec[['activityId','name','activityType','startTimeLocal','duration',
                                       'avgHr','maxHr','calories','bmrCalories','waterEstimated']]

df_garmin_imp_cols['datetime'] = pd.to_datetime(df_garmin_imp_cols['startTimeLocal'].map(lambda x: time.strftime(
                                                        '%Y-%m-%d %H:%M:%S', time.localtime(x/1000))))

df_garmin_imp_cols['date'] = df_garmin_imp_cols['datetime'].dt.date

df_garmin_imp_cols['duration'] = df_garmin_imp_cols['duration'].astype(int)

df_garmin_imp_cols['duration_minutes'] = df_garmin_imp_cols['duration'].map(lambda x: duration_measure(x))

df_garmin_imp_cols.columns = ['activityId', 'name', 'activityType', 'startTimeLocal', 'duration',
       'avgHr', 'maxHr', 'calories', 'bmrCalories', 'waterEstimated',
       'datetime', 'date', 'duration_minutes']

In [80]:
df_garmin_imp_cols.columns

Index(['activityId', 'name', 'activityType', 'startTimeLocal', 'duration',
       'avgHr', 'maxHr', 'calories', 'bmrCalories', 'waterEstimated',
       'datetime', 'date', 'duration_minutes'],
      dtype='object')

In [79]:
df_garmin_imp_cols.head()

,activityId,name,activityType,startTimeLocal,duration,avgHr,maxHr,calories,bmrCalories,waterEstimated,datetime,date,duration_minutes
0,8959734435,Elliptical,elliptical,"1,654,428,520,000.0000",2983868,148.0000,164.0000,"2,832.4535",318.4415,298.0000,2022-06-05 07:28:40,2022-06-05,49
0,8952433395,Cardio,indoor_cardio,"1,654,338,511,000.0000",144330,84.0000,89.0000,37.7102,12.5701,14.0000,2022-06-04 06:28:31,2022-06-04,2
0,8917024003,Cardio,indoor_cardio,"1,653,822,025,000.0000",6977541,121.0000,151.0000,"3,456.7665",745.8236,697.0000,2022-05-29 07:00:25,2022-05-29,116
0,8899731934,Cardio,indoor_cardio,"1,653,591,638,000.0000",3719052,153.0000,188.0000,"3,230.5054",398.0519,371.0000,2022-05-26 15:00:38,2022-05-26,61
0,8896332721,Cardio,indoor_cardio,"1,653,548,724,000.0000",6806884,136.0000,173.0000,"4,156.4998",729.0635,680.0000,2022-05-26 03:05:24,2022-05-26,113
